<a href="https://colab.research.google.com/github/tonycico0912/ChatGPT_Python/blob/main/%EA%B5%AD%EB%82%B4%EC%A3%BC%EC%8B%9D%EB%8D%B0%EC%9D%B4%ED%84%B0%EC%88%98%EC%A7%91.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install beautifulsoup4


In [4]:
import requests as rq  # requests 라이브러리를 rq라는 이름으로 가져옴
from bs4 import BeautifulSoup  # BeautifulSoup 라이브러리를 가져옴 : 이 라이브러리는 HTML을 파싱하고 정보를 추출하는데 사용됨

url = 'https://finance.naver.com/sise/sise_deposit.nhn'  # 스크래핑할 웹페이지의 URL을 정의
data = rq.get(url)  # requests의 get 함수를 사용하여 웹페이지의 HTML을 가져옴
data_html = BeautifulSoup(data.content)  # 가져온 HTML을 BeautifulSoup 객체로 변환하여 파싱할 수 있게 함

# 'div.subtop_sise_graph2 > ul.subtop_chart_note > li > span.tah' CSS 선택자를 사용하여 웹페이지에서 특정 요소를 선택
# select_one 함수는 CSS 선택자에 일치하는 첫 번째 요소를 반환
parse_day = data_html.select_one('div.subtop_sise_graph2 > ul.subtop_chart_note > li > span.tah').text

print(parse_day)  # 선택한 요소의 텍스트를 출력

import re  # re 모듈을 가져옴. re 모듈은 정규 표현식을 사용하여 문자열을 처리하는데 사용

# re.findall 함수를 사용하여 parse_day 문자열에서 숫자를 모두 찾음 '[0-9]+'는 하나 이상의 숫자를 의미하는 정규 표현식임
biz_day = re.findall('[0-9]+', parse_day)

# ''.join 함수를 사용하여 찾은 숫자들을 하나의 문자열로 합침
biz_day = ''.join(biz_day)

print(biz_day)  # 합친 문자열을 출력



  |  2023.07.06
20230706


In [10]:
import requests as rq  # 웹 요청을 보내기 위한 requests 라이브러리를 가져옴
from io import BytesIO  # 바이트 스트림을 처리하기 위한 io 라이브러리의 BytesIO를 가져옴
import pandas as pd  # 데이터 처리를 위한 pandas 라이브러리를 가져옴

###############################################################################
# KOSPI PART
###############################################################################

gen_otp_url = 'http://data.krx.co.kr/comm/fileDn/GenerateOTP/generate.cmd'  # OTP를 생성하는 URL을 정의

# OTP 생성 요청에 필요한 파라미터를 정의
gen_otp_stk = {
    'mktId': 'STK',
    'trdDd': biz_day,
    'money': '1',
    'csvxls_isNo': 'false',
    'name': 'fileDown',
    'url': 'dbms/MDC/STAT/standard/MDCSTAT03901'
}

# 요청 헤더를 정의.  Referer 헤더는 요청이 어디서 시작되었는지를 나타냄
headers = {'Referer': 'http://data.krx.co.kr/contents/MDC/MDI/mdiLoader'}

# OTP 생성 URL에 POST 요청을 보냄.  요청의 본문에는 gen_otp_stk 딕셔너리를, 헤더에는 headers 딕셔너리를 사용
otp_stk = rq.post(gen_otp_url, gen_otp_stk, headers=headers).text

print(otp_stk)  # 생성된 OTP를 출력

down_url = 'http://data.krx.co.kr/comm/fileDn/download_csv/download.cmd'  # 데이터를 다운로드하는 URL을 정의

# 다운로드 URL에 POST 요청을 보냄  요청의 본문에는 생성된 OTP를, 헤더에는 headers 딕셔너리를 사용
down_sector_stk = rq.post(down_url, {'code': otp_stk}, headers=headers)

# 다운로드한 데이터를 바이트 스트림으로 변환하고, 그 스트림을 pandas의 read_csv 함수에 전달하여 DataFrame으로 읽음
# 데이터의 인코딩은 'EUC-KR'임
sector_stk = pd.read_csv(BytesIO(down_sector_stk.content), encoding='EUC-KR')

sector_stk.head()  # DataFrame의 처음 5행을 출력.
sector_stk  # 전체 DataFrame을 출력



###############################################################################
# KOSDAQ PART
###############################################################################

# OTP 생성 요청에 필요한 파라미터를 정의. 이번에는 'mktId'를 'KSQ'로 설정하여 코스닥 데이터를 다운로드.
gen_otp_ksq = {
    'mktId': 'KSQ',  # 코스닥 입력
    'trdDd': biz_day,
    'money': '1',
    'csvxls_isNo': 'false',
    'name': 'fileDown',
    'url': 'dbms/MDC/STAT/standard/MDCSTAT03901'
}

# OTP 생성 URL에 POST 요청을 보냄. 요청의 본문에는 gen_otp_ksq 딕셔너리를, 헤더에는 headers 딕셔너리를 사용.
otp_ksq = rq.post(gen_otp_url, gen_otp_ksq, headers=headers).text

# 다운로드 URL에 POST 요청을 보냄. 요청의 본문에는 생성된 OTP를, 헤더에는 headers 딕셔너리를 사용
down_sector_ksq = rq.post(down_url, {'code': otp_ksq}, headers=headers)

# 다운로드한 데이터를 바이트 스트림으로 변환하고, 그 스트림을 pandas의 read_csv 함수에 전달하여 DataFrame으로 읽음
# 데이터의 인코딩은 'EUC-KR'
sector_ksq = pd.read_csv(BytesIO(down_sector_ksq.content), encoding='EUC-KR')

sector_ksq.head()  # DataFrame의 처음 5행을 출력

# pd.concat 함수를 사용하여 sector_stk DataFrame과 sector_ksq DataFrame을 연결.
# reset_index 함수를 사용하여 인덱스를 재설정함.  drop=True는 기존 인덱스를 삭제하라는 의미.
krx_sector = pd.concat([sector_stk, sector_ksq]).reset_index(drop=True)

# '종목명' 열의 값에서 앞뒤 공백을 제거함. str.strip 함수는 문자열의 앞뒤 공백을 제거하는 함수임
krx_sector['종목명'] = krx_sector['종목명'].str.strip()

# '기준일' 열에 biz_day 값을 설정함  이렇게 하면, 모든 행의 '기준일' 값이 biz_day 값으로 설정됨
krx_sector['기준일'] = biz_day

krx_sector.head()  # DataFrame의 처음 5행을 출력함
krx_sector


PxldVNq82TWyLSb3hI2XSdS8Zw6Cytlr5AJ//gZBgEQRtSksuLS7Bnxpl86F7dAOkunw9BBwugQaSjGAcH15eZldSFIiFMVfhJGPzsNJfHMtBgM+EFJCxYg3zco1gIgRZqIo4cIzoURnTI8+MmkJ4m8vFLhSKmM794gFu+ThsO31lY4woqehX8j6OlXFDcfHdV4NbYo4+D2Rwcfj24VnU3Zpq3ik/Dyw3FdyOXhJkBI=


,종목코드,종목명,시장구분,업종명,종가,대비,등락률,시가총액,기준일
0,095570,AJ네트웍스,KOSPI,서비스업,4275,-115,-2.62,200165311125,20230706
1,006840,AK홀딩스,KOSPI,기타금융,19080,-240,-1.24,252763463880,20230706
2,027410,BGF,KOSPI,기타금융,3715,-85,-2.24,355587878565,20230706
3,282330,BGF리테일,KOSPI,유통업,166600,-1200,-0.72,2879498739600,20230706
4,138930,BNK금융지주,KOSPI,기타금융,6730,-130,-1.90,2193544205580,20230706
...,...,...,...,...,...,...,...,...,...
2601,024060,흥구석유,KOSDAQ,유통,5360,-70,-1.29,80400000000,20230706
2602,010240,흥국,KOSDAQ,기계·장비,6550,-230,-3.39,80713658800,20230706
2603,189980,흥국에프엔비,KOSDAQ,음식료·담배,2750,-85,-3.00,110379024250,20230706
2604,037440,희림,KOSDAQ,기타서비스,7800,-230,-2.86,108595305000,20230706
